In [19]:
%%javascript
var width = window.innerWidth || document.documentElement.clientWidth || document.body.clientWidth;
var height = window.innerHeight || document.documentElement.clientHeight || document.body.clientHeight;

IPython.notebook.kernel.execute("windowSize = (" + width + "," + height + ")");
// suitable for small screens
nbpresent.mode.tree.set(
    ["app", "theme-manager", "themes", "my-theme"], 
    {
    palette: {
        "blue": { id: "blue", rgb: [0, 153, 204] },
        "black": { id: "black", rgb: [0, 0, 0] },
        "white": { id: "white", rgb: [255, 255, 255] },
        "red": { id: "red", rgb: [240, 32, 32] },
        "gray": { id: "gray", rgb: [128, 128, 128] },
    },
    backgrounds: {
        "my-background": {
            "background-color": "white"
        }
    },
    "text-base": {
        "font-family": "Georgia",
        "font-size": 2.5
    },
    rules: {
        h1: {
            "font-size": 5.5,
            color: "blue",
            "text-align": "center"
        },
        h2: {
            "font-size": 3,
            color: "blue",
            "text-align": "center"
        },
        h3: {
            "font-size": 3,
            color: "black",
        },
        "ul li": {
            "font-size": 2.5,
            color: "black"
        },
        "ul li ul li": {
            "font-size": 2.0,
            color: "black"
        },
        "code": {
            "font-size": 1.6,
        },
        "pre": {
            "font-size": 1.6,
        }
    }
});

<IPython.core.display.Javascript object>

# Multi-String BWTs

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L23Library.png" width="600px">

<p style="text-align: right; clear: right;">1</p>

# MSBWT

A BWT containing a ***string collection*** instead of just a single string

* Earliest: Mantaci et al. (2005), used concatenation approach
* Bauer et al. (2011) - proposed version we will discuss today

Analogy:

* Instead of searching for a substring within a single book, search every book of a library
    - Each book has it's own text, suffix array, and end-of-text delimiter
    - Searching allows us to find how many times a substring appears and in which texts

Bioinformatics?
* Search all genomes? You could, but that's not the main application.
* Search multiple chromosomes of an organism? You should, but even that is not the killer app

<p style="text-align: right; clear: right;">2</p>

# Naive Construction

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L23MSBWTExample.png" width="400px" style="float: right; margin: 0px 50px;">
* Create all rotations for all strings in the collection

* Sort all rotations together (Suffix Array)

* Store the predecessor of each suffix

* Strings are “cyclic”

* The predecessor is always from the same string

* Impossible to “jump” from one string to another

* Strings can have different lengths

<p style="text-align: right; clear: right;">3</p>

# MSBWT's FM-index

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L23MSBWTFMindex.png" width="500px" style="float: right; margin: 0px 50px;">
Identical Definition
* Find k-mer “CA”
* Initialize to full range ('')
* lo, hi = 0, 10
* Find occurrences of 'A'
   - lo = Offset['A'] + FMindex[lo]['A'] = 2 + 0 = 2
   - hi = Offset['A'] + FMindex[hi]['A'] = 2 + 5 = 7

* Find occurrences of “CA”
    - lo = Offset['C'] + FMindex[lo]['C'] = 7 + 0 = 7
    - hi = Offset['C'] + FMindex[hi]['C'] = 7 + 2 = 9


* Searching and extracting suffixes are identical to a BWT
<p style="text-align: right; clear: right;">4</p>

# Extra Slide
<pre style="font-weight: 500; font-size: 150%">
                                        FM-index
    String1       Sorted     MSBWT        $  A  C
    <span style="color: red;">ACCA&dollar;</span>         <span style="color: red;">&dollar;ACCA</span>        <span style="color: red;">A</span>      0:  0  0  0
    <span style="color: red;">CCA&dollar;A</span>         <span style="color: blue;">&dollar;CAAA</span>        <span style="color: blue;">A</span>      1:  0  1  0
    <span style="color: red;">CA&dollar;AC</span>         <span style="color: red;">A&dollar;ACC</span>        <span style="color: red;">C</span>      2:  0  2  0
    <span style="color: red;">A&dollar;ACC</span>         <span style="color: blue;">A&dollar;CAA</span>        <span style="color: blue;">A</span>      3:  0  2  1
    <span style="color: red;">&dollar;ACCA</span>         <span style="color: blue;">AA&dollar;CA</span>        <span style="color: blue;">A</span>      4:  0  3  1
                  <span style="color: blue;">AAA&dollar;C</span>        <span style="color: blue;">C</span>      5:  0  4  1
    String2       <span style="color: red;">ACCA&dollar;</span>        <span style="color: red;">&dollar;</span>      6:  0  4  2
    <span style="color: blue;">CAAA&dollar;</span>         <span style="color: red;">CA&dollar;AC</span>        <span style="color: red;">C</span>      7:  1  4  2
    <span style="color: blue;">AAA&dollar;C</span>         <span style="color: blue;">CAAA&dollar;</span>        <span style="color: blue;">&dollar;</span>      8:  1  4  3
    <span style="color: blue;">AA&dollar;CA</span>         <span style="color: red;">CCA&dollar;A</span>        <span style="color: red;">A</span>      9:  2  4  3
    <span style="color: blue;">A&dollar;CAA</span>                            10:  2  5  3
    <span style="color: blue;">&dollar;CAAA</span>                        Offset:  0  2  7
</pre>



# Incremental MSBWT Construction

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L23IncrementalMSBWT.png" width="400px" style="float: right; margin: 0px 50px 0px 50px;">
* A key tool missing from the BWTs toolbox--<br> *adding new strings to an existing msBWT*


* You could reconstruct the suffix array of the msBWT using suffix(i, fmindex) for all i, and then insert the suffixes of the new string.
* Variant of find(); Find the insertion point of new string's j<sup>th</sup> suffix, s<sub>j</sub>
* Add last character to msBWT
* Update the FMindex

<p style="text-align: right; clear: right;">5</p>

In [20]:
def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].iteritems()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

def inverseBWT(bwt):
    # initialize the table from t
    table = ['' for c in bwt]
    for j in xrange(len(bwt)):
        #insert the BWT as the first column
        table = sorted([c+table[i] for i, c in enumerate(bwt)])
    #return the row that ends with ‘$’
    return table[bwt.index('$')]

def BWT(t):
    # create a sorted list of all cyclic suffixes of t
    rotation = sorted([t[i:]+t[:i] for i in xrange(len(t))])
    # concatenate the last symbols from each suffix
    return ''.join(r[-1] for r in rotation)

def recoverSuffix(i, BWT, FMIndex, Offset):
    suffix = ''
    c = BWT[i]
    predec = Offset[c] + FMIndex[i][c]
    suffix = c + suffix
    while (predec != i):
        c = BWT[predec]
        predec = Offset[c] + FMIndex[predec][c]
        suffix = c + suffix
    return suffix

def findBWT(pattern, FMIndex, Offset):
    lo = 0
    hi = len(FMIndex) - 1
    for symbol in reversed(pattern):
        lo = Offset[symbol] + FMIndex[lo][symbol]
        hi = Offset[symbol] + FMIndex[hi][symbol]
    return lo, hi

bwt = "TGAG$TGC$AAA$AA"
fm, off = FMIndex(bwt)
for i in xrange(len(bwt)):
    print "%2d: %s" % (i, recoverSuffix(i, bwt, fm, off))

new = "TATA$"
inserts = []
for i in xrange(len(new)):
    l, h = findBWT(new[i:]+new[:i],fm,off)
    inserts.append((h,new[i-1]))

for i, c in sorted(inserts, reverse=True):
    bwt = bwt[:i] + c + bwt[i:]

print bwt

 0: $ACAT
 1: $ATAG
 2: $GAGA
 3: A$GAG
 4: ACAT$
 5: AG$AT
 6: AGA$G
 7: AT$AC
 8: ATAG$
 9: CAT$A
10: G$ATA
11: GA$GA
12: GAGA$
13: T$ACA
14: TAG$A
TGAAGT$TGCT$AAA$AAA$


# Merging msBWTs

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L23MergeMSBWT.png" width="200px" style="float: right; margin: 0px 150px 0px 50px;">
* BETTER YET! Rather than inserting new strings, build a BWT of the new strings and merge the new and old BWTs
* Suffixes of BTWs are already sorted
* BTWs are interleaved
* In the worse case (ties) the entire suffix must be considered, but general the longest common prefix of suffixes is smaller
* Minimal overhead
* Well suited for divide an conquer approaches (like merge sort)
* Easy to merge multiple data sets! 
* Compression improves!

<p style="text-align: right; clear: right;">6</p>

# Merging Steps

* msBWT merging alternates between sorting and interleaving
<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L23MergeSteps.png" width="700px">

<p style="text-align: right; clear: right;">7</p>

# In Python

In [21]:
def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].iteritems()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

def recoverSuffix(i, BWT, FMIndex, Offset):
    suffix = ''
    c = BWT[i]
    predec = Offset[c] + FMIndex[i][c]
    suffix = c + suffix
    while (predec != i):
        c = BWT[predec]
        predec = Offset[c] + FMIndex[predec][c]
        suffix = c + suffix
    return suffix

In [22]:
def mergeBWT(bwt1, bwt2):
    interleave = [(c, 0) for c in bwt1] + [(c, 1) for c in bwt2]
    passes = min(len(bwt1), len(bwt2))
    for p in xrange(passes):
        i, j = 0, 0
        nextInterleave = []
        for c, k in sorted(interleave, key=lambda x: x[0]):
            if (k == 0):
                b = bwt1[i]
                i += 1
            else:
                b = bwt2[j]
                j += 1
            nextInterleave.append((b, k))
        if (nextInterleave == interleave):
            break
        interleave = nextInterleave
    return ''.join([c for c, k in interleave])

bwt1 = "TG$TC$AAAA"
bwt2 = "AAGTGTA$A$"
bwt12 = mergeBWT(bwt1, bwt2)
print bwt12
FM, Offset = FMIndex(bwt12)
for i in xrange(len(bwt12)):
    j = (i>>2)+(i&3)*(len(bwt12)/4)
    print "%2d: %s" % (j, recoverSuffix(j,bwt12,FM,Offset)), "\n" if (i % 4 == 3) else "",

TGAAGT$TGCT$AAA$AAA$
 0: $ACAT   5: A$TAT  10: ATA$T  15: GAGA$ 
 1: $ATAG   6: ACAT$  11: ATAG$  16: T$ACA 
 2: $GAGA   7: AG$AT  12: CAT$A  17: TA$TA 
 3: $TATA   8: AGA$G  13: G$ATA  18: TAG$A 
 4: A$GAG   9: AT$AC  14: GA$GA  19: TATA$ 


<p style="text-align: right; clear: right;">8</p>

# MSBWT Applications

* Instead of building a BWT of a reference genome, build a MSBWT of every sequenced reads
* Arbitrary exact-match k-mer queries
* O(k) time
* Enables fast searches/counting
* Recover an arbitrary read of length L from MSBWT
* O(L) time
* Enables extraction of user-selected reads

<p style="text-align: right; clear: right;">9</p>

# Compression of high-throughput sequencing

* Using Run-length encoding again
* Reasons we expect compression:
* True genomic repeats: gene families, long repeats, etc.
* Over-sampling: 30x coverage means we expect 30 copies of every k-mer pattern
* Sequencing errors may break up runs
* Technical errors may cause biases for or against a particular pattern
* Real Mouse DNA-seq:
   * 368654191 &times; 151 &times; 2 = ~112 Giga-bases
   * Compresses to ~15.3 GB using RLE (1.09 bits/base)
* Real Mouse RNA-seq:
   * ~8.9 Giga-bases
   * ~1.2 GB using RLE (1.05 bits/base)
   
<p style="text-align: right; clear: right;">10</p>

# K-mer Search & Extraction

Basic Use:
<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L23KmerSearch.png">
<p style="text-align: center;">Green: query k-mer. Red: forward reads. Blue: reverse-complement reads.</p>

* Search for all reads with a given k-mer
* Extract all reads with that k-mer and its reverse-complement
* Build a consensus
<p style="text-align: right; clear: right;">11</p>

# Reference-based Searches

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L23Egr3.png" width="500px" style="float: right;">
* Given a reference genome and region of that genome
* Split reference into k-mers
* Count the abundance of each k-mer and plot
* Fast - O(k) time per k-mer
* Similar to a post-alignment pileup

<p style="text-align: right; clear: right;">12</p>

# Reference Correction

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L23RefCorrection.png" width="800px">

<p style="text-align: right; clear: right;">13</p>

# Summary

* Burrows-Wheeler Transform
    - Permutation of characters that represents a suffix array
    - Run-length encoded for compression

* FM-index
    - Derived from BWT
    - Exploits LF-mapping property
    - O(k) search time for arbitrary k-mer, independent of BWT's size
    - Used in many fast aligners

* MSBWT
    - Applies to string collections
    - Enables database-like access to reads via k-mer searches
    
<p style="text-align: right; clear: right;">14</p>

# Next Time

### Sequence Alignment
<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/AlignCartoon.png">

<p style="text-align: right; clear: right;">15</p>